In [41]:
# MNIST and softmax
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms

In [42]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [43]:
# parameters
learning_rate = 0.5
batch_size = 10

In [44]:
#MNIST datasets
mnist_train = dsets.MNIST(root ='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [45]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [46]:
w1 = torch.nn.Parameter(torch.Tensor(784,30)).to(device)
b1 = torch.nn.Parameter(torch.Tensor(30)).to(device)
w2 = torch.nn.Parameter(torch.Tensor(30,10)).to(device)
b2 = torch.nn.Parameter(torch.Tensor(10)).to(device)

In [47]:
torch.nn.init.normal_(w1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(w2)
torch.nn.init.normal_(b2)

tensor([ 0.1009, -0.2272, -0.1022, -0.9369,  0.6552, -0.8914,  0.1989,  0.7264,
         0.3932, -0.5963], device='cuda:0', grad_fn=<ToCopyBackward0>)

In [48]:
def sigmoid(x):
  return 1.0/(1.0 + torch.exp(-x))

In [49]:
def sigmoid_prime(x):
  return sigmoid(x) * (1-sigmoid(x))

In [50]:
X_test = mnist_test.test_data.view(-1,28*28).float().to(device)[:1000]
Y_test = mnist_test.test_labels.to(device)[:1000]
i=0
while not i == 10000:
  for X,Y in data_loader:
    i+=1

    X = X.view(-1,28*28).to(device)
    Y = torch.zeros((batch_size, 10)).scatter_(1, Y.unsqueeze(1),1).to(device)
    l1 = torch.add(torch.matmul(X,w1),b1)
    a1 = sigmoid(l1)
    l2 = torch.add(torch.matmul(a1, w2), b2)
    y_pred = sigmoid(l2)

    diff = y_pred - Y
    #layer 2
    d_l2 = diff * sigmoid_prime(l2)
    d_b2 = d_l2
    d_w2 = torch.matmul(torch.transpose(a1,0,1),d_l2)
    #layer 1
    d_a1 = torch.matmul(d_l2, torch.transpose(w2,0,1))
    d_l1 = d_a1*sigmoid_prime(l1)
    d_b1 = d_l1
    d_w1 = torch.matmul(torch.transpose(X,0,1), d_l1)

    #Weight update
    w1 = w1 - learning_rate * d_w1
    b1 = b1 - learning_rate * torch.mean(d_b1, 0)
    w2 = w2 - learning_rate * d_w2
    b2 = b2 - learning_rate * torch.mean(d_b2, 0)


    if i % 1000 == 0:
      l1 = torch.add(torch.mm(X_test, w1),b1)
      a1 = sigmoid(l1)
      l2 = torch.add(torch.mm(a1, w2), b2)
      y_pred = sigmoid(l2)
      acct_mat = torch.argmax(y_pred, 1) == Y_test
      acct_res = acct_mat.sum()
      print(acct_res.item())

    if i == 10000:
      break

810
849
882
894
902
902
887
910
912
896
